In [1]:
import pyspark

vehicle_counter_DF = spark.read.csv('/home/madhavi/per-vehicle-records-2019-10-31.csv', inferSchema = True, header = True)

In [8]:
type(vehicle_counter_DF)

pyspark.sql.dataframe.DataFrame

In [14]:
vehicle_counter_DF.count()

4389299

In [158]:
vehicle_counter_DF.select("cosit").distinct().count()

325

In [16]:
# Q1 Show total number of counts by each vehicle class.

vehicle_counter_DF.groupBy('classname').count().show()

+---------+-------+
|classname|  count|
+---------+-------+
|      CAR|3472965|
|  HGV_ART| 216978|
|      BUS|  29347|
|  HGV_RIG| 135202|
|     null|    396|
|  CARAVAN|  21224|
|      LGV| 498505|
|    MBIKE|  14682|
+---------+-------+



In [10]:
# Q2  Showtop5highestnumberofcountsbyvehicleclass. 

from pyspark.sql.functions import desc
vehicle_counter_DF.groupBy('classname').count().sort(desc("count")).show(5)

+---------+-------+
|classname|  count|
+---------+-------+
|      CAR|3472965|
|      LGV| 498505|
|  HGV_ART| 216978|
|  HGV_RIG| 135202|
|      BUS|  29347|
+---------+-------+
only showing top 5 rows



In [146]:
#3 show the total number of counts by the largest sized (length) vehicle.
vehicle_counter_DF.groupBy('length').count().sort(desc("length")).show(5)

+------+-----+
|length|count|
+------+-----+
| 40.47|    1|
| 39.46|    1|
| 36.77|    1|
| 34.17|    1|
| 32.45|    1|
+------+-----+
only showing top 5 rows



In [90]:
#Q4  How many vehicles were counted on straddlelane? 

vehicle_counter_DF.filter(vehicle_counter_DF["straddlelane"]>=0).count()


4389299

In [98]:
#Q5 Compute average speed (for each counter) of vehicles with respect to their class
from pyspark.sql.functions import asc
vehicle_counter_DF.groupby ("class").agg({"speed":"avg"}).sort(asc("class")).show()

+-----+-----------------+
|class|       avg(speed)|
+-----+-----------------+
|    0|81.18964646464646|
|    1|75.41983381010762|
|    2|87.99111496948547|
|    3|90.35929148153001|
|    4| 79.0626980611306|
|    5|80.11806925933027|
|    6|81.93572758528522|
|    7|  80.509602336977|
+-----+-----------------+



In [142]:
#Q6 Combine the date and time fields (year, month, day, hour, minute,
#seconds, millisecond) into one variable (e.g. YYY-MM-DD-HH-MM-SSMMM) and call it timestamp.

from pyspark.sql.functions import concat, col, lit
df2 = vehicle_counter_DF.withColumn(
    'Timestamp',concat(col("year"), lit("-"), col("month"), lit("-"), col("day"), lit(" "), col("hour")
                                , lit(":"), col("minute"), lit(":"), col("second"), lit(":"), col("millisecond")))
                          
df2.show(2)

+-----+----+-----+---+----+------+------+-----------+-----------+----+--------+------------+----------------+-----+---------+------+-------+----+-----+------+-----------+--------+------------+-------------+-----------+------------+-------------------+
|cosit|year|month|day|hour|minute|second|millisecond|minuteofday|lane|lanename|straddlelane|straddlelanename|class|classname|length|headway| gap|speed|weight|temperature|duration|validitycode|numberofaxles|axleweights|axlespacings|          Timestamp|
+-----+----+-----+---+----+------+------+-----------+-----------+----+--------+------------+----------------+-----+---------+------+-------+----+-----+------+-----------+--------+------------+-------------+-----------+------------+-------------------+
|  997|2019|   10| 31|   0|    15|     1|          0|         15|   1|   Test1|           0|            null|    2|      CAR|   5.0|   3.57|3.21| 69.0|   0.0|        0.0|       0|           0|            0|       null|        null|2019-10-31 0:

In [147]:
import pyspark.sql.functions as F
df3 = df2.withColumn('TIME_timestamp',F.to_timestamp("Timestamp", "yyyy-MM-dd HH:mm:ss:mi"))
df3.show(2)

+-----+----+-----+---+----+------+------+-----------+-----------+----+--------+------------+----------------+-----+---------+------+-------+----+-----+------+-----------+--------+------------+-------------+-----------+------------+-------------------+--------------+
|cosit|year|month|day|hour|minute|second|millisecond|minuteofday|lane|lanename|straddlelane|straddlelanename|class|classname|length|headway| gap|speed|weight|temperature|duration|validitycode|numberofaxles|axleweights|axlespacings|          Timestamp|TIME_timestamp|
+-----+----+-----+---+----+------+------+-----------+-----------+----+--------+------------+----------------+-----+---------+------+-------+----+-----+------+-----------+--------+------------+-------------+-----------+------------+-------------------+--------------+
|  997|2019|   10| 31|   0|    15|     1|          0|         15|   1|   Test1|           0|            null|    2|      CAR|   5.0|   3.57|3.21| 69.0|   0.0|        0.0|       0|           0|       

In [166]:
#7. List the top 3 busiest roads in Ireland (sites)
vehicle_counter_DF.groupby ("cosit").agg({"headway":"avg"}).sort(asc("cosit")).show(3)

+-----+------------------+
|cosit|      avg(headway)|
+-----+------------------+
|  997|3.1881781953787263|
| 1011| 8.330199114368558|
| 1012| 5.421058468563986|
+-----+------------------+
only showing top 3 rows



In [171]:
#8  Show total number of counts by each number of axles class.
vehicle_counter_DF.groupBy('numberofaxles').count().sort("count",ascending=False).show(5)

+-------------+-------+
|numberofaxles|  count|
+-------------+-------+
|            0|4201123|
|            2| 172814|
|            5|   6280|
|            6|   3640|
|            4|   3542|
+-------------+-------+
only showing top 5 rows



In [167]:
#9 average duration of vehicles with respect to their class.
vehicle_counter_DF.groupBy("class").avg("duration").show(5)

+-----+------------------+
|class|     avg(duration)|
+-----+------------------+
|    1| 4.878150115788039|
|    6|35.441911161500244|
|    3|10.263937172144713|
|    5|27.505621218621027|
|    4|26.821140150611647|
+-----+------------------+
only showing top 5 rows



In [163]:
#10 number vehicles travelled on lane 1
vehicle_counter_DF.filter(vehicle_counter_DF["lane"]==1).count()


1119899